# Models

Define our project:

In [95]:
stty -echo
mkdir workspace4
django-admin startproject myproject workspace4
cd workspace4

Create our webapp:

In [96]:
python manage.py startapp models_app

Django uses a Model-View architecture; we will modify `view.py` and `models.py` in this tutorial.  
We begin by defining a model.  Django advises against using an `__init__` method, so we've created a create method instead:

In [97]:
cat << EOF > models_app/models.py
from django.db import models

class Match(models.Model):
    homeTeam = models.CharField(max_length=20)
    homeScore = models.IntegerField(default=0)
    awayTeam = models.CharField(max_length=20)
    awayScore = models.IntegerField(default=0)

    def create(self, home, score1, away, score2):
        self.homeTeam = home
        self.homeScore = score1
        self.awayTeam = away
        self.awayScore = score2
        
    def __str__(self):
        return f"{self.homeTeam} {self.homeScore}-{self.awayScore} {self.awayTeam}"
EOF

Now perform migrations for the default apps

In [98]:
python manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

The `migrate` command has created a new class in `models/apps.py` 

In [99]:
cat models_app/apps.py

from django.apps import AppConfig


class ModelsAppConfig(AppConfig):
    default_auto_field = 'django.db.models.BigAutoField'
    name = 'models_app'


`makemigrations` basically generates the SQL commands for preinstalled apps (which can be viewed in installed apps in settings.py) and your newly created apps' model which you add in installed apps.It does not execute those commands in your database file. So tables aren't created after makemigrations.

After applying makemigrations you can see those SQL commands with `sqlmigrate` which shows all the SQL commands that have been generated by makemigrations.

`migrate` executes those SQL commands in database file. So after executing migrate, all the tables of your installed apps are created in your database file.  

However, our model has not yet been registered, so `the table corresponding to our model has not yet been created`.  

This `ModelsConfig` class has to be published in the global `project/settings.py` file.  

In [100]:
sed -i -e "/INSTALLED_APPS/ a \    'models_app.apps.ModelsAppConfig'," myproject/settings.py

The relevant part of this file now reads:

In [101]:
sed -n '/^INSTALLED_APPS/,/^]/p' myproject/settings.py

INSTALLED_APPS = [
    'models_app.apps.ModelsAppConfig',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]


Now tell Django that you’ve made some changes to your models (in this case, you’ve made new ones) and that you’d like the changes to be stored as a migration:

In [102]:
python manage.py makemigrations models_app

Migrations for 'models_app':
  models_app/migrations/0001_initial.py
    + Create model Match


Let's take a moment to see the migrations Django has created:

In [103]:
python manage.py sqlmigrate models_app 0001

BEGIN;
--
-- Create model Match
--
CREATE TABLE "models_app_match" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "homeTeam" varchar(20) NOT NULL, "homeScore" integer NOT NULL, "awayTeam" varchar(20) NOT NULL, "awayScore" integer NOT NULL);
COMMIT;


Now, run migrate to create the `models_app_match` table in your database:

In [104]:
python manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, models_app, sessions
Running migrations:
  Applying models_app.0001_initial... OK


Check that the new table has been created using `dbshell`:

In [105]:
python manage.py dbshell << EOF
SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name = 'models_app_match';
EOF

models_app_match


Use django shell to add data to the `models_app_match` table

In [106]:
# Access model field values via Python attributes.
python manage.py shell << EOF
from models_app.models import Match  # Import the model classes we just wrote.

m1 = Match()
m2 = Match()
m3 = Match()
m1.create("Red", 2, "Blue", 1)
m2.create("Green", 5, "White", 0)
m3.create("Red", 3, "Green", 1)
m1.save()
m2.save()
m3.save()

for match in Match.objects.all():
    print(match)
EOF

Red 2-1 Blue
Green 5-0 White
Red 3-1 Green


Now use `dbshell` to see the same data:

In [107]:
python manage.py dbshell << EOF
SELECT 
    *
FROM 
    models_app_match
EOF

1|Red|2|Blue|1
2|Green|5|White|0
3|Red|3|Green|1


Now it's time to create views so that we can see our results in a browser.

In [108]:
cat << EOF >> models_app/views.py
from models_app.models import Match


def resultsView(request):
    matches = []
    try:
        for match in Match.objects.all():
            matches.append(match)
    except:
        raise Http404("Problems!!")
    
    context = {
        'matches':matches, 
    }
    return render(request, "resultsTemplate.html", context)
EOF

Now create the template.  I haven't used an external stylesheet this time, but consult the previous notebook to see how to set one up.

In [109]:
mkdir -p models_app/templates
cat << EOF > models_app/templates/resultsTemplate.html
<style>
body {background-color: powderblue;}
table, th, td {
  border: 1px solid;
}
</style>

<h1>Results</h1>
<table>
{% for match in matches %}
    <tr><td>{{ match }}</td></tr>
{% endfor %}
</table>
EOF

Set up the `urls.py` to direct us to the view:

In [110]:
# add url pattern to point at a new view
cat << EOF > models_app/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('results/', views.resultsView, name='resultsView'),
]
EOF

Now modify the master urls file:

In [111]:
cat << EOF > myproject/urls.py
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('models_app/', include('models_app.urls')),
]
EOF

Let's use `firefox` to see the app:

In [112]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

[1] 98435


In [113]:
firefox http://localhost:7000/models_app/results

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
December 16, 2024 - 12:37:20
Django version 5.1.3, using settings 'myproject.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

firefox http://localhost:7000/models_app/results^J


clean up:
* kill the server
* remove workspace

In [ ]:
cd ..
fuser -k 7000/tcp
rm -r workspace4